In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

This notebook is an updated version of a notebook contributed by [Fabian Hirschmann](https://github.com/fhirschmann).

# E2E ML on GCP: MLOps stage 2 : experimentation: get started with Vertex AI Training for R using R Kernel

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_r_using_r_kernel.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_r_using_r_kernel.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_r_using_r_kernel.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                        
</table>

## Overview

This example demonstrates how to train and deploy R models with `Vertex AI` using an R kernel -- such as in `Vertex AI Workbench Notebooks`.

### Objective

In this tutorial, you learn how to use `Vertex AI`, using an R kernel, for training and deploying an R custom model.

This tutorial uses the following Google Cloud ML services:

- `Vertex AI Training`
- `Vertex AI Prediction`
- `Vertex AI Model` resource
- `Vertex AI Endpoint` resource

The steps performed include:

- Create a custom R training script
- Create a custom R serving script
- Create a custom R deployment (serving) container.
- Train the model using `Vertex AI` custom training.
- Create an `Endpoint` resouce.
- Deploy the `Model` resource (trained R model) to the `Endpoint` resource.
- Make an online prediction.


### Dataset

The dataset used for this tutorial is [California Housing Dataset](https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html). The data contains information from the 1990 California census. The data set is publicly available from Cloud Storage at `gs://cloud-samples-data/ai-platform-unified/datasets/tabular/california-housing-tabular-regression.csv`. The dataset is used to train a Random Forest regressor to predict a median housing price, given a longitude and lattitude along with data from the corresponding census block group. A block group is the smallest geographical unit for which the U.S. Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people).


### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Vertex AI Workbench Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* R
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with R and Python 3

The Cloud Storage guide to [Setting up a Python development environment](https://cloud.google.com/python/setup) and the [Jupyter installation guide](https://jupyter.org/install) provide detailed instructions for meeting these requirements. The following steps provide a condensed set of instructions:

1. [Install and initialize the SDK](https://cloud.google.com/sdk/docs/).

2. [Install R]().

3. [Install virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) and create a virtual environment that uses R.  Activate the virtual environment.

4. To install Jupyter, run `pip3 install jupyter` on the command-line in a terminal shell.

5. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

6. Open this notebook in the Jupyter Notebook Dashboard.


#### Define helper functions for this Notebook

First, you define some helper functions used throughout this tutorial.

- `sh`: Executes the specified command shell.
- `display_file`: Displays the contents of the specified file.

In [ ]:
library(glue)
library(IRdisplay)

sh <- function(cmd, args = c(), intern = FALSE) {
    if (is.null(args)) {
        cmd <- glue(cmd)
        s <- strsplit(cmd, " ")[[1]]
        cmd <- s[1]
        args <- s[2:length(s)]
    }
    ret <- system2(cmd, args, stdout = TRUE, stderr = TRUE)
    if ("errmsg" %in% attributes(attributes(ret))$names) cat(attr(ret, "errmsg"), "\n")
    if (intern) return(ret) else cat(paste(ret, collapse = "\n"))
}

display_file <- function(filename) {
    body <- sh(glue("pygmentize -g {filename} -f html -P full -O style=default"), intern = TRUE)
    IRdisplay::display_html(paste("<div style='text-align: left'>", paste(body, collapse="\n"), "</div>", collapse="\n"))
}

## Installation

Install the following packages to execute this notebook.

In [ ]:
required_packages < -c("reticulate", "glue", "httr")
install.packages(setdiff(required_packages, rownames(installed.packages())))

sh("pip3 install --upgrade google-cloud-aiplatform -q")

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com) and the [Artifact Registry API](https://console.cloud.google.com/flows/enableapi?apiid=artifactregistry.googleapis.com).

1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Have the project ID autodetected or enter it below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID < -"[your-project-id]"  # @param {type:"string"}

In [ ]:
# Get your Google Cloud project ID from gcloud
if (Sys.getenv("IS_TESTING") == "") {
    if (PROJECT_ID == "[your-project-id]") {
      PROJECT_ID <- sh("gcloud config list --format 'value(core.project)' 2>/dev/null", intern = TRUE)
    }
    cat("Project ID:", PROJECT_ID, "\n")
}

In [ ]:
sh("gcloud config set project {PROJECT_ID}")

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION <- "[your-region]"  # @param {type: "string"}

if (REGION == "[your-region]") {
    REGION <- "us-central1"
}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
TIMESTAMP = format(Sys.time(), "%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
# The Google Cloud Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK <- file.exists("/opt/deeplearning/metadata/env_version")
IS_GOOGLE_COLAB <- (system("python -c 'import google.colab'") != 0)

GOOGLE_APPLICATION_CREDENTIALS <- ""  # @param {type:"string"}

# If on Vertex AI Workbench Notebooks, then don't execute this code
if (!IS_WORKBENCH_NOTEBOOK) {
    # Replace the string below with the path to your service account key and run this cell
    # to authenticate your GCP account.
    Sys.setenv("GOOGLE_APPLICATION_CREDENTIALS" = GOOGLE_APPLICATION_CREDENTIALS)
}

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex AI SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [ ]:
BUCKET_NAME < -"[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI < -paste0("gs://", BUCKET_NAME)

In [ ]:
if (BUCKET_NAME == "" || BUCKET_NAME == "[your-bucket-name]") {
    BUCKET_NAME = paste0(PROJECT_ID, "aip-", TIMESTAMP)
    BUCKET_URI = paste0("gs://", BUCKET_NAME)
}

BUCKET_URI

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
sh("gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}")

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
sh("gsutil ls -al {BUCKET_URI}")

### Import libraries and define constants

Import and initialize the `reticulate` R package to interface with the Vertex AI SDK, which is written in Python.

In [ ]:
library(reticulate)
library(glue)
library(httr)
library(IRdisplay)

use_python(Sys.which("python3"))

aiplatform <- import("google.cloud.aiplatform")

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aiplatform$init(project = PROJECT_ID, location = REGION, staging_bucket = BUCKET_URI)

## Introduction to Google Artifact Registry

The `Google Artifact Registry` is a service for storing and managing artifacts in private repositories, including container images, Helm charts, and language packages. It is the recommended container image registry for Google Cloud.

Learn more about [Quick start for Docker](https://cloud.google.com/artifact-registry/docs/docker/quickstart)

### Enable Artifact Registry API

First, you must enable the Artifact Registry API service for your project.

Learn more about [Enabling service](https://cloud.google.com/artifact-registry/docs/enable-service).

In [ ]:
sh("gcloud services enable artifactregistry.googleapis.com")

## Create a private Docker repository

Your first step is to create your own Docker repository in Google Artifact Registry.

1. Run the `gcloud artifacts repositories create` command to create a new Docker repository with your region with the description "docker repository".

2. Run the `gcloud artifacts repositories list` command to verify that your repository was created.

In [ ]:
PRIVATE_REPO < -"my-docker-repo"

sh(
    'gcloud artifacts repositories create {PRIVATE_REPO} --repository-format=docker --location={REGION} --description="Docker repository"'
)

sh("gcloud artifacts repositories list")

### Configure authentication to your private repo

Before you push or pull container images, configure Docker to use the `gcloud` command-line tool to authenticate requests to `Artifact Registry` for your region.

In [ ]:
sh("gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet")

### Create Dockerfile

The docker file for your custom container is built on top of the Deep Learning container -- the same container that is also used for Vertex AI Workbench. In addition, you add two R scripts for model training and serving, respectively.

In [ ]:
IMAGE_NAME < -"vertex-r"  # @param {type:"string"}
IMAGE_TAG < -"latest"  # @param {type:"string"}
IMAGE_URI < -glue(
    "{REGION}-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/{IMAGE_NAME}:{IMAGE_TAG}"
)

dir.create("src", showWarnings=FALSE)

In [ ]:
Dockerfile = cat("
# filename: Dockerfile - container specifications for using R in Vertex AI
FROM gcr.io/deeplearning-platform-release/r-cpu.4-1:latest

WORKDIR /root

COPY train.R /root/train.R
COPY serve.R /root/serve.R

# Install Fortran
RUN apt-get update
RUN apt-get install gfortran -yy

# Install R packages
RUN Rscript -e \"install.packages('plumber')\"
RUN Rscript -e \"install.packages('randomForest')\"

EXPOSE 8080
", file = "src/Dockerfile")

display_file("src/Dockerfile")

### Create the training script

Next, create the file `train.R`, which is used to train your R model. The script trains a `randomForest` model on the California Housing dataset. Vertex AI sets environment variables that you can utilize, and since this script uses a Vertex AI managed dataset, data splits are performed by Vertex AI and the script receives environment variables pointing to the training, test, and validation sets.

The trained model artifacts are then stored to your Cloud Storage bucket.

In [ ]:
cat('
#!/usr/bin/env Rscript
# filename: train.R - train a Random Forest model on Vertex AI Managed Dataset
library(tidyverse)
library(data.table)
library(randomForest)
Sys.getenv()

# The GCP Project ID
project_id <- Sys.getenv("CLOUD_ML_PROJECT_ID")

# The GCP Region
location <- Sys.getenv("CLOUD_ML_REGION")

# The Cloud Storage URI to upload the trained model artifact to
model_dir <- Sys.getenv("AIP_MODEL_DIR")

# Next, you create directories to download our training, validation, and test set into.
dir.create("training")
dir.create("validation")
dir.create("test")

# You download the Vertex AI managed data sets into the container environment locally.
system2("gsutil", c("cp", Sys.getenv("AIP_TRAINING_DATA_URI"), "training/"))
system2("gsutil", c("cp", Sys.getenv("AIP_VALIDATION_DATA_URI"), "validation/"))
system2("gsutil", c("cp", Sys.getenv("AIP_TEST_DATA_URI"), "test/"))

# For each data set, you may receive one or more CSV files that you will read into data frames.
training_df <- list.files("training", full.names = TRUE) %>% map_df(~fread(.))
validation_df <- list.files("validation", full.names = TRUE) %>% map_df(~fread(.))
test_df <- list.files("test", full.names = TRUE) %>% map_df(~fread(.))

print("Starting Model Training")
rf <- randomForest(median_house_value ~ ., data=training_df, ntree=100)
rf

saveRDS(rf, "rf.rds")
system2("gsutil", c("cp", "rf.rds", model_dir))
', file = "src/train.R")

display_file("src/train.R")

### Create the serving script

Next, create the file `serve.R`, which is used for serving your R model. The script downloads the model artifact from Cloud Storage, loads the model artifacts, and listens for prediction requests on port `8080`.

In [ ]:
cat('
#!/usr/bin/env Rscript
# filename: serve.R - serve predictions from a Random Forest model
Sys.getenv()
library(plumber)

system2("gsutil", c("cp", "-r", Sys.getenv("AIP_STORAGE_URI"), "."))
system("du -a .")

rf <- readRDS("artifacts/rf.rds")
library(randomForest)

predict_route <- function(req, res) {
    print("Handling prediction request")
    df <- as.data.frame(req$body$instances)
    preds <- predict(rf, df)
    return(list(predictions=preds))
}

print("Staring Serving")

pr() %>%
    pr_get(Sys.getenv("AIP_HEALTH_ROUTE"), function() "OK") %>%
    pr_post(Sys.getenv("AIP_PREDICT_ROUTE"), predict_route) %>%
    pr_run(host = "0.0.0.0", port=as.integer(Sys.getenv("AIP_HTTP_PORT", 8080)))
', file = "src/serve.R")

display_file("src/serve.R")

### Build the Docker container

Next, you build the Docker container image on Cloud Build -- the serverless CI/CD platform.

*Note:* Building the Docker container image may take 10 to 15 minutes.

In [ ]:
sh("cd src && gcloud builds submit --region={REGION} --tag={IMAGE_URI} --timeout=1h")

## Create Vertex AI Managed Dataset

You create a Vertex AI Managed dataset to have Vertex AI take care of the data set split. This is optional, and alternatively you may want to pass the URI to the data set via environment variables.

In [ ]:
data_uri <- "gs://cloud-samples-data/ai-platform-unified/datasets/tabular/california-housing-tabular-regression.csv"

dataset <- aiplatform$TabularDataset$create(
    display_name = "California Housing Dataset",
    gcs_source = data_uri
)

## Create a Vertex AI custom training job

The custom training job wraps the training process by creating an instance of your container image and executing `train.R` for model training and `serve.R` for model serving.

*Note:* You use the same custom container for both training and serving.

In [ ]:
job <- aiplatform$CustomContainerTrainingJob(
    display_name = "vertex-r",
    container_uri = IMAGE_URI,
    command = c("Rscript", "train.R"),
    model_serving_container_command = c("Rscript", "serve.R"),
    model_serving_container_image_uri = IMAGE_URI
)

### Execute the custom training job


To train the model, you call the method `run()`, with a machine type that is sufficient in resources to train a machine learning model on your dataset. For this tutorial, you use a `n1-standard-4` VM instance.

In [ ]:
model <- job$run(
    dataset=dataset,
    model_display_name = "vertex-r-model",
    machine_type = "n1-standard-4"
)

model$display_name
model$resource_name
model$uri

## Creating an `Endpoint` resource

You create an `Endpoint` resource using the `Endpoint.create()` method. At a minimum, you specify the display name for the endpoint. Optionally, you can specify the project and location (region); otherwise the settings are inherited by the values you set when you initialized the Vertex AI SDK with the `init()` method.

In this example, the following parameters are specified:

- `display_name`: A human readable name for the `Endpoint` resource.
- `project`: Your project ID.
- `location`: Your region.
- `labels`: (optional) User defined metadata for the `Endpoint` in the form of key/value pairs.

This method returns an `Endpoint` object.

Learn more about [Vertex AI Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api).

In [ ]:
endpoint <- aiplatform$Endpoint$create(
    display_name = "California Housing Endpoint",
    project = PROJECT_ID,
    location = REGION
)

endpoint

## Deploying `Model` resources to an `Endpoint` resource.

You can deploy one of more `Vertex AI Model` resource instances to the same endpoint. Each `Vertex AI Model` resource that is deployed will have its own deployment container for the serving binary. 

Next, you deploy the `Vertex AI Model` resource to a `Vertex AI Endpoint` resource. The `Vertex AI Model` resource already has defined for it the deployment container image. To deploy, you specify the following additional configuration settings:

- The machine type.
- The (if any) type and number of GPUs.
- Static, manual or auto-scaling of VM instances.

In this example, you deploy the model with the minimal amount of specified parameters, as follows:

- `model`: The `Model` resource.
- `deployed_model_displayed_name`: The human readable name for the deployed model instance.
- `machine_type`: The machine type for each VM instance.

Do to the requirements to provision the resource, this may take upto a few minutes.

*Note:* For this example, you specified the R deployment container in the previous step of uploading the model artifacts to a `Vertex AI Model` resource.

In [ ]:
model$deploy(endpoint = endpoint, machine_type = "n1-standard-4")

### Test the deployed model

Now that your `Model` resource is deployed to an `Endpoint` resource, you can do online predictions by sending prediction requests to the Endpoint resource.

#### Create example data

First, you create some example data to test making a prediction request to your deployed model. You use five JSON-encoded example data points (without the label `median_house_value`) from the original data file in `data_uri`.


In [ ]:
df <- read.csv(text=sh("gsutil cat {data_uri}", intern = TRUE))
head(df, 5)

instances = list(instances=head(df[, names(df) != "median_house_value"], 5))
instances

json_instances = toJSON(instances)
json_instances

#### Make a prediction request

Finally, you make a prediction request with your example data. In this example, you use the REST API (e.g., Curl) to make the prediction request.

In [ ]:
url < -glue(
    "https://{REGION}-aiplatform.googleapis.com/v1/{endpoint$resource_name}:predict"
)
access_token < -sh("gcloud auth print-access-token", intern=TRUE)

sh(
    "curl",
    c(
        "--tr-encoding",
        "-s",
        "-X POST",
        glue("-H 'Authorization: Bearer {access_token}'"),
        "-H 'Content-Type: application/jsoin'",
        url,
        glue("-d {json_instances}"),
    ),
)

## Clean up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
delete_bucket = FALSE

endpoint$undeploy_all()
endpoint$delete()
dataset$delete()
model$delete()
job$delete()

sh("rm -rf src")

# remove Docker image
sh("docker rmi {IMAGE_URI}")

if (delete_bucket == TRUE) sh("gsutil -m rm -r {BUCKET_URI}")